<a href="https://colab.research.google.com/github/Brandon-Bernal-Alarcon/Notas/blob/main/Machine%20Learning/Aprendizaje%20Supervisado/Introduccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aprendizaje Supervisado**


Este notebook forma parte de mis notas del curso “Applied Machine Learning in Python” de la Universidad de Michigan, el objetivo es introducir los conceptos fundamentales del aprendizaje supervisado, combinando teoría esencial con ejemplos prácticos en Python usando **scikit-learn**.

Este documento sirve tanto como material de estudio como referencia futura propio.

## **Objetivos**

Entender qué es el aprendizaje supervisado y cómo se diferencia de otros enfoques de Machine Learning.

Conocer los dos grandes tipos de problemas: regresión y clasificación.

Introducir el concepto de conjunto de entrenamiento y prueba.

Comprender la idea de generalización, underfitting y overfitting.

## **¿Qué es el aprendizaje supervisado?**

El aprendizaje supervisado es un enfoque de Machine Learning en el cual el modelo aprende a partir de datos etiquetados, cada observación del conjunto de datos contiene:

- Un conjunto de variables predictoras (features) $X$

- Una variable objetivo o etiqueta

El objetivo del modelo es aprender una función:

$$\hat{y} = f(X)$$



que permita predecir correctamente la salida para nuevos datos no vistos.

In [2]:
#@title Librerías
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Definimos $X$ como el conjunto de caracteristicas de las entidades (matriz de caracteristicas), las cuales se tomarán como entradas para el predictor, mientras que Y es el vector de etiquetas o valores objetivo.

In [3]:
# Este es el dataset con el que vamos a trabajar
frutas = pd.read_table('/content/fruit_data_with_colors.txt')
frutas.head()

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [4]:
#@title Representacion de entidades

np.set_printoptions(precision=2) # Limita a 2 decimales los resultados

frutas = pd.read_table('/content/fruit_data_with_colors.txt')

caracteristicas = ['height', 'width', 'mass', 'color_score']
etiqueta = ['fruit_label']

X_frutas = frutas[caracteristicas]
y_frutas = frutas[etiqueta]
nombres = ['apple', 'mandarin', 'orange', 'lemon']

X_frutas_2d = frutas[['height', 'width']]
y_frutas_2d = frutas['fruit_label']

## **Conjuntos de entrenamiento y prueba**

El dataset se divide en dos partes, un **conjunto de entrenamiento** que se usa para aprender los parámetros del modelo y un *conjunto de prueba* se usa para evaluar qué tan bien generaliza el modelo.
<br><br>

Convención típica:

75% entrenamiento

25% prueba <br><br>


Esta separación es clave para detectar problemas como el sobreajuste, se toma el conjunto original de caracteristicas $X$ y las etiquetas de $y$  para hacer una partición del conjunto de entrenamiento

En scikit-learn:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_frutas, y_frutas, random_state=0)

## **Normalizacion de los datos**

Antes de entrenar el modelo, se normalizan (escalan) las características, lo cual es fundamental para k-NN y muchos otros algoritmos.

*MinMaxScaler()* transforma cada característica para que quede en un rango fijo, por defecto [0,1] mediante: <br>

$$x' = \frac{x-x_{min}}{x_{max}-x_{min}}$$

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##**Entrenamiento del modelo (Model Fitting)**

Un modelo en scikit-learn se conoce como un estimador (classifier o regressor).

El proceso es:

- Se crea el modelo

- Se llama al método fit(X_train, y_train)

- El modelo ajusta sus parámetros internos usando los datos de entrenamiento

- Después del ajuste, el modelo queda listo para: Evaluación y Predicción de nuevos datos

In [7]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train_scaled, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

## **Evaluación del modelo**

Una vez entrenado, el modelo se evalúa usando el conjunto de prueba, se introduce la precisión (accuracy) como métrica básica, que es la proporción de predicciones correctas sobre el total.

En general, la comparación entre la precisión en entrenamiento y en prueba permite evaluar la capacidad de generalización de un modelo.

- Si ambas precisiones son altas y similares, el modelo generaliza bien.

- Si la precisión en entrenamiento es alta pero baja en prueba, hay sobreajuste.

- Si ambas precisiones son bajas, el modelo presenta subajuste.

Estas métricas deben interpretarse considerando el tamaño y la complejidad del dataset.


In [8]:
print('Accuracy de K-NN classifier en el set de entrenamiento es: {:.2f}'
     .format(knn.score(X_train_scaled, y_train)))
print('Accuracy de K-NN classifier en el set de prueba es: {:.2f}'
     .format(knn.score(X_test_scaled, y_test)))

Accuracy de K-NN classifier en el set de entrenamiento es: 0.95
Accuracy de K-NN classifier en el set de prueba es: 1.00



El clasificador k-NN con k=5 obtiene una precisión del 95% en el conjunto de entrenamiento y del 100% en el conjunto de prueba, esto indica que el modelo generaliza bien para este dataset, ya que las clases están claramente separadas y las características utilizadas son informativas, no se observan signos de sobreajuste en esta partición de los datos.

## **Predicción sobre datos nuevos**

El modelo entrenado puede utilizarse para **Predecir** la etiqueta de instancias que no formaron parte del entrenamiento

Esto se hace mediante el método **predict()** Este paso representa el objetivo final del aprendizaje supervisado

In [9]:
datos = [5.5, 2.2, 10, 0.70]

ejemplo = pd.DataFrame([datos],
    columns=caracteristicas)

ejemplo_escalado = scaler.transform(ejemplo)
print('La prediccion para', datos, ' es ',
          nombres[knn.predict(ejemplo_escalado)[0]-1])

La prediccion para [5.5, 2.2, 10, 0.7]  es  mandarin


## **Tipos de problemas de aprendizaje supervisado**

**a) Clasificación**

El valor objetivo es discreto y se puede dividir en:

- Clasificación binaria: se clasifica en 0 o 1, como transacción fraudulenta vs. no fraudulenta

- Clasificación multiclase, no se limita a un sí o un no, sino que puedes tener una gama de opciones, como tipo de fruta (manzana, naranja, etc.)

- Clasificación multietiqueta
Ejemplo: asignar múltiples temas a una página web

**b) Regresión**

El valor objetivo es continuo, por ejemplo se puede usar para la predicción del precio de una casa, la predicción de ingresos, altura, temperatura, etc.

La naturaleza del valor objetivo determina si el problema es de clasificación o regresión.

## **Subajuste, buen ajuste y sobreajuste**

Dependiendo de la complejidad del modelo y de los datos disponibles, pueden presentarse tres situaciones:

- **Subajuste (Underfitting)**:  
  El modelo es demasiado simple y no logra capturar los patrones básicos de los datos. Tiene bajo rendimiento tanto en entrenamiento como en prueba.

- **Buen ajuste (Good fit)**:  
  El modelo logra capturar la tendencia general de los datos, ignorando el ruido. Presenta buen desempeño tanto en entrenamiento como en prueba.

- **Sobreajuste (Overfitting)**:  
  El modelo es demasiado complejo y se ajusta excesivamente a los datos de entrenamiento, capturando variaciones locales que no se repiten en datos nuevos. Su desempeño en prueba es pobre.

El objetivo es encontrar un equilibrio entre simplicidad y complejidad que permita una buena generalización.


## **Sobreajuste en regresión y clasificación**

El fenómeno del sobreajuste puede observarse tanto en problemas de regresión como de clasificación.

En regresión, un modelo demasiado simple (por ejemplo, una recta) puede no capturar la relación real entre las variables, mientras que un modelo demasiado complejo puede ajustarse perfectamente a los datos de entrenamiento pero fallar en datos nuevos.

En clasificación, el problema se refleja en los límites de decisión. Un modelo muy simple puede no separar correctamente las clases, mientras que un modelo demasiado complejo puede crear fronteras muy irregulares que separan perfectamente los datos de entrenamiento, pero que son muy sensibles al ruido y a valores atípicos.
